In [1]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
pose_5 = loadmat("PIE dataset/Pose05_64x64.mat")
pose_7 = loadmat("PIE dataset/Pose07_64x64.mat")
pose_9 = loadmat("PIE dataset/Pose09_64x64.mat")
pose_27 = loadmat("PIE dataset/Pose27_64x64.mat")
pose_29 = loadmat("PIE dataset/Pose29_64x64.mat")

In [3]:
X_all = np.concatenate([pose_5['fea'], pose_7['fea'], pose_9['fea'], pose_27['fea'], pose_29['fea'] ]).reshape(-1, 64, 64)

In [4]:
pose_5

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon May 08 13:10:30 2017',
 '__version__': '1.0',
 '__globals__': [],
 'isTest': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]]),
 'fea': array([[ 8,  9,  9, ..., 35, 36, 42],
        [ 6,  7,  7, ..., 38, 32, 31],
        [ 8,  8,  6, ..., 26, 25, 23],
        ...,
        [11, 10,  8, ..., 36, 40, 42],
        [ 9, 11,  9, ..., 29, 31, 36],
        [12,  9,  9, ..., 29, 34, 38]], dtype=uint8),
 'gnd': array([[ 1],
        [ 1],
        [ 1],
        ...,
        [68],
        [68],
        [68]], dtype=uint8)}

In [5]:
y_all = np.concatenate([pose_5['gnd'], pose_7['gnd'], pose_9['gnd'], pose_27['gnd'], pose_29['gnd'] ])

In [6]:
isTest_all = np.concatenate([pose_5['isTest'], pose_7['isTest'], pose_9['isTest'], pose_27['isTest'], pose_29['isTest'] ])

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

In [9]:
class FaceDataSet(Dataset):
    def __init__(self, X, y, isTest, data_type='train', transform=None):
        self.X = X
        self.y = y
        self.isTest = isTest
        self.data_type = data_type
        self.transform = transform
        X_train = []
        y_train = []
        X_test = []
        y_test = []
        for i in range(self.X.shape[0]):
            if self.isTest[i]==1:
                X_test.append(self.X[i])
                y_test.append(self.y[i])
            else:
                X_train.append(self.X[i])
                y_train.append(self.y[i])
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train)
        self.X_test = np.array(X_test)
        self.y_test = np.array(y_test)
        
        
        
    def __len__(self):
        if self.data_type=='train':
            #print((self.X_train.shape[0], self.y_train.shape[0]))
            return self.X_train.shape[0]
        else:
            #print((self.X_test.shape[0], self.y_test.shape[0]))
            return self.X_test.shape[0]
    
    def __getitem__(self, idx):
        if self.data_type=='train':
            face = Image.fromarray(self.X_train[idx])
            face = face.convert('RGB')
            label = self.y_train[idx][0] - 1
        else:
            face = Image.fromarray(self.X_test[idx])
            face = face.convert('RGB')
            label = self.y_test[idx][0] - 1

        if self.transform:
            face = self.transform(face)
        sample = {'face': face, 'label': label}
        
        return sample
        

In [10]:
data_transforms = {
    'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()]),
    'test': transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
}

face_datasets = {x: FaceDataSet(X_all, y_all, isTest_all, data_type=x, transform=data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(face_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(face_datasets[x]) for x in ['train', 'test']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [19]:
import time
import copy


In [29]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch: {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for batch in dataloaders[phase]:
                face = batch['face'].to(device)
                labels = batch['label'].to(device)
                #print(face.shape)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(face)

                    _, preds = torch.max(outputs, 1)
                    #return (outputs, labels)
                    loss = criterion(outputs, labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * face.shape[0]
                running_corrects += torch.sum(preds == labels.data)
                
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        time_elapsed = time.time() - since
        print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
        
        if phase=='test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time() - since
    print("Training completed in {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed%60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model
                    
                
                

In [30]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 68)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [31]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

Epoch: 0/4
----------
test Loss: 0.2769 Acc: 0.9257
Time elapsed 2m 16s

Epoch: 1/4
----------
test Loss: 0.0824 Acc: 0.9768
Time elapsed 4m 31s

Epoch: 2/4
----------
test Loss: 0.0228 Acc: 0.9946
Time elapsed 6m 45s

Epoch: 3/4
----------
test Loss: 0.0189 Acc: 0.9946
Time elapsed 9m 1s

Epoch: 4/4
----------
test Loss: 0.0114 Acc: 0.9961
Time elapsed 11m 16s

Training completed in 11m 16s
Best acc: 0.9961


In [32]:
torch.save(model_ft, 'pie/best.model')